<a href="https://colab.research.google.com/github/thetsuthetsu/PythonSamples/blob/main/PyTorch_Lightning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pytorch_lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.6/774.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 11.3 MB/s eta 0:00:00


In [2]:
import pytorch_lightning as pl

In [3]:
pl.__version__

'2.1.0'

In [4]:
import torch

In [5]:
import torch.nn as nn
import torch.nn.functional as F

In [6]:
# データ準備
from sklearn.datasets import load_iris
iris = load_iris()
x = iris['data']
t = iris['target']

In [7]:
# PytorchのTensor型へ変換
x = torch.tensor(x, dtype=torch.float32)
t = torch.tensor(t, dtype=torch.int64)

In [8]:
# 入力値と目標値をまとめる
dataset = torch.utils.data.TensorDataset(x,t)

In [9]:
# 各データのサンプル数を決定
# train : val : test = 60 : 20 : 20
n_train = int(len(dataset) * 0.6)
n_val = int(len(dataset) * 0.2)
n_test = len(dataset) - n_train - n_val

In [11]:
# データセットの分割
torch.manual_seed(0)
train, val, test = torch.utils.data.random_split(dataset, [n_train, n_val, n_test])

In [12]:
# バッチサイズ (目安：学習サンプル数の1/10～1/100)
batch_size = 10

# Data Loaderを用意
# 訓練データのみshuffleをTrue
train_loader = torch.utils.data.DataLoader(train, batch_size, shuffle=True, drop_last=True)
val_loader = torch.utils.data.DataLoader(val, batch_size)
test_loader = torch.utils.data.DataLoader(test, batch_size)

In [50]:
## ネットワークの定義
# 4 -> 4 -> 3
class Net(pl.LightningModule):
  def __init__(self):
    super().__init__()
    self.fc1 = nn.Linear(4,4)
    self.fc2 = nn.Linear(4,3)

  def forward(self, x):
    h = self.fc1(x)
    h = F.relu(h)
    h = self.fc2(h)
    return h

  def training_step(self, batch, batch_idx):
    x,t = batch
    y = self(x)
    loss = F.cross_entropy(y,t)
    # ログを追加
    self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
    self.log('train_acc', accuracy(y,t, task="multiclass", num_classes=3), on_step=True, on_epoch=True, prog_bar=True)
    return loss

  # 検証データに対する処理
  def validation_step(self, batch, batch_idx):
    x, t = batch
    y = self(x)
    loss = F.cross_entropy(y,t)
    # ログを追加
    self.log('val_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
    self.log('val_acc', accuracy(y,t, task="multiclass", num_classes=3), on_step=True, on_epoch=True, prog_bar=True)
    return loss

  # テストデータに対する処理
  def test_step(self, batch, batch_idx):
    x, t = batch
    y = self(x)
    loss = F.cross_entropy(y,t)
    # ログを追加
    self.log('test_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
    self.log('test_acc', accuracy(y,t, task="multiclass", num_classes=3), on_step=True, on_epoch=True, prog_bar=True)
    return loss

  def configure_optimizers(self):
    optimizer = torch.optim.SGD(self.parameters(), lr=0.01)
    return optimizer


In [51]:
pl.seed_everything(0)

net = Net()

INFO:lightning_fabric.utilities.seed:Seed set to 0


In [52]:
net

Net(
  (fc1): Linear(in_features=4, out_features=4, bias=True)
  (fc2): Linear(in_features=4, out_features=3, bias=True)
)

In [53]:
# 学習
trainer = pl.Trainer(max_epochs=30, accelerator="auto", deterministic=True)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [54]:
trainer.fit(net, train_loader, val_loader)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name | Type   | Params
--------------------------------
0 | fc1  | Linear | 20    
1 | fc2  | Linear | 15    
--------------------------------
35        Trainable params
0         Non-trainable params
35        Total params
0.000     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=1` in the `DataLoader` to improve performance.
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=1` in the `DataLoader` to improve performance.
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loops/fit_loop.py:293: The number of training batches (9) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=30` reached.


In [55]:
# 評価指標
trainer.callback_metrics

{'train_loss': tensor(0.7549),
 'train_loss_step': tensor(0.9137),
 'train_acc': tensor(0.8556),
 'train_acc_step': tensor(0.8000),
 'val_loss': tensor(0.6431),
 'val_loss_epoch': tensor(0.6431),
 'val_acc': tensor(0.8667),
 'val_acc_epoch': tensor(0.8667),
 'train_loss_epoch': tensor(0.7549),
 'train_acc_epoch': tensor(0.8556)}

In [56]:
# テストデータの評価
results = trainer.test(dataloaders=test_loader)

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:145: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/lightning_logs/version_4/checkpoints/epoch=29-step=270.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from the checkpoint at /content/lightning_logs/version_4/checkpoints/epoch=29-step=270.ckpt
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers

Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      test_acc_epoch       │    0.9333333373069763     │
│      test_loss_epoch      │    0.7351360321044922     │
└───────────────────────────┴───────────────────────────┘

In [57]:
results

[{'test_loss_epoch': 0.7351360321044922, 'test_acc_epoch': 0.9333333373069763}]